Telco Customer Churn Prediction Project

1. Veri Yükleme ve Ön İnceleme (EDA)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

df = pd.read_csv("../data/data.csv")

2. Veri Ön İşleme (Preprocessing)

In [2]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
#df.head()


In [3]:
df['TotalCharges'] = df['TotalCharges'].fillna(0)
df.isnull().sum()
print(df['TotalCharges'].dtype)


float64


In [4]:
df['Churn'].value_counts(normalize=True)


Churn
No     0.73463
Yes    0.26537
Name: proportion, dtype: float64

Elimizde Soyle Bir Tablo Var:  
-%73 Mutlu Müşteri (No): Kalmışlar.  
-%26 Küskün Müşteri (Yes): Gitmişler.  
-Bizim burada yapmamız gereken şeylerden en önemlisi modelimizi eğitirken doğruluk oranımızın /73 üstü olması gerekiyor.  
->Çünkü biz hiçbir şey yapmasak model  Yes / No dese %73 tutturacak zaten.  

In [5]:
df.drop('customerID', axis=1,inplace=True)

In [6]:
df.groupby('gender')['Churn'].value_counts(normalize=True)

gender  Churn
Female  No       0.730791
        Yes      0.269209
Male    No       0.738397
        Yes      0.261603
Name: proportion, dtype: float64

Kodlama yaparken açıkcası cinsiyet de işe yarıyor mu diye kendimi  sorguladım bu yüzden bir üst satırdaki kodu yazdım..  
Aradaki Fark çok az olduğu için 'genderi' i de verimizden atıyorum.  

In [7]:
df.drop('gender',axis=1,inplace=True)

In [8]:
df.groupby('Churn')[['tenure','MonthlyCharges']].mean(numeric_only=True)


,tenure,MonthlyCharges
Churn,,
No,37.569965,61.265124
Yes,17.979133,74.441332


Gidenlerin faturası, kalanlara göre %20 daha pahalı.

In [9]:
df.groupby('Churn')['Contract'].value_counts(normalize=True)

Churn  Contract      
No     Month-to-month    0.429068
       Two year          0.318322
       One year          0.252609
Yes    Month-to-month    0.885500
       One year          0.088818
       Two year          0.025682
Name: proportion, dtype: float64

Gidenlerin (Yes) %88'i "Month-to-month" (Taahhütsüz) sözleşme kullanıyormuş.  
Gidenlerin sadece %2'si "Two year" (2 Yıllık) sözleşmeliymiş.  

->Modelimizi geliştirme aşamasına geçmeden önce şu analizi iyi yapmış olmamız gerekiyor..  
1.Yeni Gelmiş Olması  
2.Faturaları yüksek gelmiş  
3.Taahhütsüz olması  
Bizim şirketimizden müşterinin gitmesine neden olan sebeblerdir.  


In [10]:
new_df = df.replace({"Yes":1,"No":0})
df.head()

,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


Bilgisayarımız Yes ve no dan anlamadığı için tüm 'Yes'=1 yaptık 'No'=0 yaptık..


In [11]:
new_df.info()

<class 'pandas.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   SeniorCitizen     7043 non-null   int64  
 1   Partner           7043 non-null   object 
 2   Dependents        7043 non-null   object 
 3   tenure            7043 non-null   int64  
 4   PhoneService      7043 non-null   object 
 5   MultipleLines     7043 non-null   object 
 6   InternetService   7043 non-null   object 
 7   OnlineSecurity    7043 non-null   object 
 8   OnlineBackup      7043 non-null   object 
 9   DeviceProtection  7043 non-null   object 
 10  TechSupport       7043 non-null   object 
 11  StreamingTV       7043 non-null   object 
 12  StreamingMovies   7043 non-null   object 
 13  Contract          7043 non-null   str    
 14  PaperlessBilling  7043 non-null   object 
 15  PaymentMethod     7043 non-null   str    
 16  MonthlyCharges    7043 non-null   float64
 17  TotalC

In [12]:
print(new_df)

      SeniorCitizen Partner Dependents  tenure PhoneService     MultipleLines  \
0                 0       1          0       1            0  No phone service   
1                 0       0          0      34            1                 0   
2                 0       0          0       2            1                 0   
3                 0       0          0      45            0  No phone service   
4                 0       0          0       2            1                 0   
...             ...     ...        ...     ...          ...               ...   
7038              0       1          1      24            1                 1   
7039              0       1          1      72            1                 1   
7040              0       1          1      11            0  No phone service   
7041              1       1          0       4            1                 1   
7042              0       0          0      66            1                 0   

     InternetService Online

Dummy Variable Trap (Kukla Değişken Tuzağı)  
Şimdi 00 ise 3.Kısım kesin 1 olur bunu herseferinde yaparsak modelin kafası karışık bu tuzağa düşmememiz gerekiyor.  

In [13]:
new_df = pd.get_dummies (new_df, drop_first=True)  #One-Hot Encoding

drop_first=True gereksiz olan sütünü atıcaktır 3 sütün yerine  2 sutunla idare edicektir.  


In [14]:
new_df.info()

<class 'pandas.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 30 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   SeniorCitizen                          7043 non-null   int64  
 1   tenure                                 7043 non-null   int64  
 2   MonthlyCharges                         7043 non-null   float64
 3   TotalCharges                           7043 non-null   float64
 4   Partner_1                              7043 non-null   bool   
 5   Dependents_1                           7043 non-null   bool   
 6   PhoneService_1                         7043 non-null   bool   
 7   MultipleLines_1                        7043 non-null   bool   
 8   MultipleLines_No phone service         7043 non-null   bool   
 9   InternetService_DSL                    7043 non-null   bool   
 10  InternetService_Fiber optic            7043 non-null   bool   
 11  OnlineSecurity_

In [15]:
y = new_df['Churn_1']
X = new_df.drop('Churn_1',axis=1)

In [16]:
X.shape  


(7043, 29)

In [17]:
y.shape

(7043,)

In [18]:
from sklearn.model_selection import train_test_split
X_train , X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)


Not: Veriyi önce ham haliyle (Scaling olmadan) modele sokup, başarısızlığı göreceğiz. Ardından Standartlaştırma (Scaling) yapıp farkı inceleyeceğiz.

3.Modelleme (Logistic Regression & Random Forest)

In [19]:
from sklearn.linear_model import LogisticRegression
log_R = LogisticRegression(max_iter=3000,class_weight='balanced') #Modeli cezalandırdık class_weight ile bunu daha sonradan ekledim
log_R.fit(X_train,y_train)
y_pred = log_R.predict(X_test)

c:\Users\akarf\OneDrive\Masaüstü\Musteri-Terk-Analizi\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:406: ConvergenceWarning: lbfgs failed to converge after 3000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=3000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [20]:
from sklearn.metrics import accuracy_score
accuracy= accuracy_score(y_test,y_pred)
print(accuracy)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix (y_test,y_pred)
print(cm)



0.7494677075940384
[[747 289]
 [ 64 309]]


a11 = Model Dedi ki Gitmez ,Gerçekte Gitmedi yani TruePoz.    
a12 = Model Dedi ki Gider ,Gerçekte Gitmedi yani  FalsePoz    
a21 = Model Dedi ki Gider , Gerçekte Gitti yani 223 kişi Gitti Modelimiz İyi iş çıkardı.TrueNeg.   
a22 = Model Dedi ki Gitmez, Gerçekte Gitti yani 150 kişi elimizden kaçtı Model Kötü iş yaptı.TruePoz.  

Şimdi bak bir şirket çalışanı olarak kendini düşün LineerReg. Kullanıyoruz ve biz 223 kişinin gideceğini tahmin ettik bunu önlemek için şirkete ilk sunacağımız şey %10 veya %20 indirim sunmak.Bu şekilde şirketi kar'a koyabiliriz.
İkinci kısıma gelelim şirketin ruhu duymadan 150 kişi elimizden kaçtı bu durumda ne yapacağız?

Toplam kaç kişi gitti?--> 373 kişi (150 + 223).  
Sen kaçını yakaladın?--> 223 kişiyi.  
Matematiksel olarak: 223 / 373 = %59.7  
Yani giden müşterilerin %40'ını kaçırıyoruz.  

Şimdi ilk görevimiz bizim sayıları 0-1 arasında sıkıştırmamız model burda yanılıyor çünkü ondan dolayı 150 kişi haberimiz olmadan gitti  


In [21]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


Bak aşağıdaki kodumuzda biz bir junior değil senior gibi mühendislik takasını (trade-off) yaptık başarımız azaldı ama elde gözle görülür bir verimiz var..

In [22]:
log_R = LogisticRegression(max_iter=3000,class_weight='balanced')
log_R.fit(X_train,y_train)
y_pred = log_R.predict(X_test)
accuracy_1 = accuracy_score(y_test,y_pred)
cm_1 = confusion_matrix(y_test,y_pred)
print(accuracy_1)
print(cm_1)

0.7480482611781405
[[746 290]
 [ 65 308]]


Fazladan  +85 Kişiyi Gördü ve Yakaladi.  
Kaçanlar Azaldı Eskiden 151 Tane idi şimdi 65.  
Ama 290 Kısmında biraz sorun oluştu model o kadar dedektif oldu ki gitmeyene de gidicek gözüyle baktı.  

Bakın az da olsa ML çalışmış biri baseline(aptal model ) den ne farkımız var der boşa uğraştık diye    yargılayabilir ama unutmayın ki aptal modelin kaçan müşteri tahmini = 0 dır.  
Biz burda 308 tane kaçan müşteri yakaladık başarı oranımız %74.8 olsada 308 kişiye güzel bir indirim sağlayıp şirketin kar elde etmesi sağlanabilir..

Kanıt Olarak Aşağıya Kod Bırakıyorum

In [23]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.92      0.72      0.81      1036
        True       0.52      0.83      0.63       373

    accuracy                           0.75      1409
   macro avg       0.72      0.77      0.72      1409
weighted avg       0.81      0.75      0.76      1409



recall'de True Değerine bakıcak olursak 100 müşteriden 83 ünü doğru tahmin ettik yakaladık..

Şimdi kaçanların neden kaçtıgını analiz edicez 

In [24]:
log_R.coef_

array([[ 0.05749907, -1.25252374, -0.85436894,  0.61394192,  0.01805553,
        -0.06853581, -0.00343904,  0.16676418,  0.00343904, -0.29025887,
         0.39658621, -0.13246729, -0.14344377, -0.01948748, -0.14344377,
         0.03454164, -0.14344377, -0.09287449, -0.14344377,  0.22013688,
        -0.14344377,  0.27389903, -0.14344377, -0.28193482, -0.64889622,
         0.15372694, -0.02692245,  0.16106539, -0.01546857]])

En yüksek POZİTİF ->Kovan Sebeb  
En düşük  NEGATİF ->Tutan Sebeb  

In [25]:
pd.Series(data=log_R.coef_[0],index=X.columns).sort_values()

tenure                                  -1.252524
MonthlyCharges                          -0.854369
Contract_Two year                       -0.648896
InternetService_DSL                     -0.290259
Contract_One year                       -0.281935
StreamingTV_No internet service         -0.143444
OnlineBackup_No internet service        -0.143444
OnlineSecurity_No internet service      -0.143444
DeviceProtection_No internet service    -0.143444
StreamingMovies_No internet service     -0.143444
TechSupport_No internet service         -0.143444
OnlineSecurity_1                        -0.132467
TechSupport_1                           -0.092874
Dependents_1                            -0.068536
PaymentMethod_Credit card (automatic)   -0.026922
OnlineBackup_1                          -0.019487
PaymentMethod_Mailed check              -0.015469
PhoneService_1                          -0.003439
MultipleLines_No phone service           0.003439
Partner_1                                0.018056


Bak burad TotalCharges Yani fatura ücreti arttıkça müşterinin net kaçtıgını görebiliyoruz..  
Artık Kadroda kalanlar yani tenurelere ise karışma öp başının üstüne koy.


In [26]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(class_weight='balanced')
rfc.fit(X_train,y_train)
y_pred = rfc.predict(X_test)
accuracy_2 = accuracy_score(y_test,y_pred)
cm_2 = confusion_matrix(y_test,y_pred)
print(accuracy_2)
print(cm_2)

0.7970191625266146
[[950  86]
 [200 173]]


Random yukarıdaki haliyle sınıfta kaldı

In [27]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(class_weight='balanced')
rfc.fit(X_train,y_train)
y_olasiliklar = rfc.predict_proba(X_test)
yeni_tahmin = (y_olasiliklar[:, 1] > 0.30)
accuracy_2 = accuracy_score(y_test,yeni_tahmin)
cm_2 = confusion_matrix(y_test,yeni_tahmin)
print(accuracy_2)
print(cm_2)

0.7629524485450674
[[813 223]
 [111 262]]


Yukarıdaki modele bakalım şimdi 815 sadık müşteriyi doğru tahmin  etti  
Tam 100 Müşteriyi Kurtardı.a21 bak  
a12 a bakarsak da   
Yaklaşık 130 tane masum müşteriye de "Sen gideceksin galiba" dedi. Ama bu bizim için sorun değil %10 luk indirim tanımlamış oldun her ihtimalle yukarıdakinden daha iyidir.  

elimzide şimdi iki model var   
Lojistik Regresyon: Agresif, çok yakalıyor ama çok hata yapıyor.  
Random Forest: Dengeli, ayar yapınca iş görüyor.  